In [1]:
# this code gets 10 levels of orderbook data every second from IBKR, must be run in live notebook during market hours, saved to a csv file on macbook desktop

In [ ]:
# Most recent code, run when market is open 
import os
import time
from ib_insync import *
import pandas as pd
import random

# Connect IB once synchronously; IB will run its own event loop internally
ib = IB()
client_id = random.randint(1, 10000)
print(f'Connecting with client ID: {client_id}')
import nest_asyncio
nest_asyncio.apply()

await ib.connectAsync('127.0.0.1', 7497, clientId=client_id)

# Define the contract for Tesla on NASDAQ
contract = Stock('TSLA', 'NASDAQ', 'USD')

# Specify number of depth rows per side (max 10 levels)
num_levels = 10

# Subscribe to market depth data (Level 2) with 10 levels per side
ticker = ib.reqMktDepth(contract, numRows=num_levels)

# Prepare CSV file path on desktop
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
csv_path = os.path.join(desktop_path, 'tesla realtime l2.csv')

# Write header for CSV
with open(csv_path, 'w') as f:
    f.write('timestamp,side,position,operation,price,size\n')

print('Starting to collect order book data every second...')

try:
    while True:
        # Collect current bid and ask depth data snapshot
        rows = []
        timestamp = pd.Timestamp.now()
        for bid in ticker.domBids:
            rows.append([timestamp, 'bid', bid.position, bid.operation, bid.price, bid.size])
        for ask in ticker.domAsks:
            rows.append([timestamp, 'ask', ask.position, ask.operation, ask.price, ask.size])

        # Append rows to CSV
        df = pd.DataFrame(rows, columns=['timestamp', 'side', 'position', 'operation', 'price', 'size'])
        df.to_csv(csv_path, mode='a', header=False, index=False)

        print(f"Appended {len(rows)} rows at {timestamp}")

        time.sleep(1)  # Wait 1 second before next snapshot

except KeyboardInterrupt:
    print('Stopping data collection...')

finally:
    ib.cancelMktDepth(contract)
    ib.disconnect()
    print('Disconnected from IB API')


Connecting with client ID: 6182
Starting to collect order book data every second...
Appended 0 rows at 2025-12-01 11:49:26.508017
Appended 0 rows at 2025-12-01 11:49:27.515409
Appended 0 rows at 2025-12-01 11:49:28.525116
Appended 0 rows at 2025-12-01 11:49:29.532085


In [ ]:
from ib_insync import *

# This starts asyncio event loop integration for Jupyter
util.startLoop()

# Connect to TWS or IB Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

# Define the TSLA option contract
tsla_option = Option('TSLA', '20231215', 220, 'C', 'SMART')

# Request market depth data (5 rows of bids and asks)
ticker = ib.reqMktDepth(tsla_option, numRows=5, isSmartDepth=True)

# Callback function for market depth updates
def onDepthUpdate():
    print("Bids:")
    for bid in ticker.domBids:
        print(bid)
    print("Asks:")
    for ask in ticker.domAsks:
        print(ask)
    print('---')

# Register callback for updates
ticker.updateEvent += onDepthUpdate

# Keep script running for a while to receive updates
import time
time.sleep(15)

# Cancel market depth subscription and disconnect
ib.cancelMktDepth(tsla_option, isSmartDepth=True)
ib.disconnect()


In [ ]:
from ib_insync import *
import pandas as pd
from IPython.display import display, clear_output

# Start the asyncio event loop in Jupyter
util.startLoop()

# Connect to IB Gateway or TWS
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=19)

# Define the TSLA option contract
tsla_option = Option('TSLA', '20251205', 430, 'C', 'SMART')

# Request market depth data (5 rows per side)
ticker = ib.reqMktDepth(tsla_option, numRows=5, isSmartDepth=True)

# Initialize a DataFrame with columns for bid and ask sizes and prices
df = pd.DataFrame(index=range(5), columns=['bidSize', 'bidPrice', 'askPrice', 'askSize'])

# Callback to update DataFrame and display it live in the notebook
def onDepthUpdate():
    bids = ticker.domBids
    asks = ticker.domAsks
    for i in range(5):
        df.loc[i, 'bidSize'] = bids[i].size if i < len(bids) else 0
        df.loc[i, 'bidPrice'] = bids[i].price if i < len(bids) else 0
        df.loc[i, 'askPrice'] = asks[i].price if i < len(asks) else 0
        df.loc[i, 'askSize'] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)

# Register callback for market depth updates
ticker.updateEvent += onDepthUpdate

# Run for 15 seconds to collect and display updates
import time
time.sleep(15)

# Cancel depth subscription and disconnect
ib.cancelMktDepth(tsla_option, isSmartDepth=True)
ib.disconnect()


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import random
from ib_insync import *

ib = IB()
client_id = random.randint(1, 10000)
async def main():
    await ib.connectAsync('127.0.0.1', 7497, clientId=client_id)
    contract = Stock('TSLA', 'SMART', 'USD')
    ticker = ib.reqMktData(contract)
    await asyncio.sleep(2)  # wait for bid/ask update
    print(f'Bid: {ticker.bid}, Ask: {ticker.ask}')
    ib.cancelMktData(ticker)
    ib.disconnect()

import asyncio
asyncio.run(main())


cancelMktData: No reqId found for contract Ticker(contract=Stock(symbol='TSLA', exchange='SMART', currency='USD'))


Bid: nan, Ask: nan


In [2]:
import nest_asyncio
import random
from ib_insync import *
import asyncio

nest_asyncio.apply()

ib = IB()
client_id = random.randint(1, 10000)

async def main():
    await ib.connectAsync('127.0.0.1', 7497, clientId=client_id)
    contract = Stock('TSLA', 'NASDAQ', 'USD')
    
    # Request market depth with 5 levels per side
    ticker = ib.reqMktDepth(contract, numRows=5)
    
    # Wait a few seconds to accumulate depth data
    await asyncio.sleep(5)
    
    print("Top 5 Bids:")
    for bid in ticker.domBids:
        print(f"Position: {bid.position}, Price: {bid.price}, Size: {bid.size}")
    
    print("\nTop 5 Asks:")
    for ask in ticker.domAsks:
        print(f"Position: {ask.position}, Price: {ask.price}, Size: {ask.size}")
    
    ib.cancelMktDepth(contract)
    ib.disconnect()

asyncio.run(main())


API connection failed: ConnectionRefusedError(61, "Connect call failed ('127.0.0.1', 7497)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 61] Connect call failed ('127.0.0.1', 7497)